In [19]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [20]:
import os
project_path = '/content/gdrive/My Drive/sw융합프로젝트(1)/'
print("프로젝트 폴더:", project_path)
print("포함된 파일 목록:", os.listdir(project_path))

프로젝트 폴더: /content/gdrive/My Drive/sw융합프로젝트(1)/
포함된 파일 목록: ['행정구역(읍면동)', '행정구역(시군구)', '인천광역시_시내버스 정류소 현황_20241231.csv', '인천광역시.geojson', '아파트(매매)_실거래가_20250512141056.csv', '연립다세대(전월세)_실거래가_20250512142423.csv', '아파트(전월세)_실거래가_20250512142547.csv', '연립다세대(매매)_실거래가_20250512142554.csv', 'multi_geocoded.csv', 'apt_geocoded.csv', '버스정류장_법정동매핑.csv', '버스정류장_법정동매핑.geojson']


# 1. 시군구 열에서 어느 동인지만 분리

In [21]:
import pandas as pd
import geopandas as gpd

apt = pd.read_csv(project_path + 'apt_geocoded.csv')
multi = pd.read_csv(project_path + 'multi_geocoded.csv')
bus = gpd.read_file(project_path + '버스정류장_법정동매핑.geojson')

In [31]:
# apt와 multi를 concat으로 합치기
merged = pd.concat([apt, multi], ignore_index=True)

# '시군구'에서 '법적동' 추출 → 'OO동' 형태의 마지막 단어만 추출
merged['법적동'] = merged['시군구'].str.extract(r'(\S+동)')

merged.head(3)

,시군구,단지명,도로명,주소,위도,경도,geometry,법적동
0,인천광역시 서구 원당동,검단신도시디에트르더힐,서로3로 255,인천광역시 서구 원당동 서로3로 255,37.601007,126.709360,POINT (126.709360468052 37.6010069478289),원당동
1,인천광역시 서구 가좌동,한신휴플러스,건지로 402,인천광역시 서구 가좌동 건지로 402,37.495266,126.684407,POINT (126.6844067696 37.4952664818856),가좌동
2,인천광역시 서구 원당동,우미린더헤리티지,이음4로 94,인천광역시 서구 원당동 이음4로 94,37.591893,126.720148,POINT (126.720148399306 37.5918933926296),원당동


In [32]:
merged["법적동"].unique()

array(['원당동', '가좌동', '청라동', '당하동', '마전동', '가정동', '오류동', '신현동', '심곡동',
       '불로동', '석남동', '검암동', '왕길동', '금곡동', '연희동', '경서동', '백석동', '공촌동',
       '대곡동'], dtype=object)

In [33]:
merged["법적동"].value_counts()

,count
법적동,
청라동,4697
당하동,3445
원당동,3060
가정동,2137
가좌동,1939
마전동,1829
석남동,1472
검암동,1241
신현동,1145


In [34]:
# geometry 생성 후 GeoDataFrame으로 변환
from shapely.geometry import Point
import geopandas as gpd

merged['geometry'] = merged.apply(lambda row: Point(row['경도'], row['위도']), axis=1)
gdf_merged = gpd.GeoDataFrame(merged, geometry='geometry', crs='EPSG:4326')

# 거리 연산을 위해 UTM 좌표계로 변환
gdf_merged = gdf_merged.to_crs(epsg=5181)

왜 UTM (예: EPSG:5181) 좌표계로 변환해야 할까?

: EPSG:4326은 위도(latitude)/경도(longitude) 기반 좌표계이고,
이는 지구의 곡률을 반영한 구형 좌표계, 이 상태로는 다음이 어려움

- 거리 계산 → 위경도는 미터 단위가 아니므로 두 점 간의 거리를 meter로 직접 계산할 수 없음

- 버퍼 생성 → 예: 반경 500m 안에 정류장이 있는지 체크


(변환 없이 거리를 구하는 지오이드 기반 거리 계산 방식 (haversine)도 있음, GeoDataFrame에서 buffer, distance 같은 공간 연산을 쓸 거면 지금 방식이 맞음)



### 아파트 or 오피스텔 중에서, 반경 500m 내에 버스정류장이 하나도 없는 곳을 찾기

In [37]:
from shapely.geometry import Point
from tqdm import tqdm
tqdm.pandas()

# merged에 아파트 + 오피스텔이 포함돼 있고, bus가 GeoDataFrame 형태로 주어졌다고 가정
# 1. merged를 GeoDataFrame으로 변환
merged['geometry'] = merged.apply(lambda row: Point(row['경도'], row['위도']), axis=1)
gdf_merged = gpd.GeoDataFrame(merged, geometry='geometry', crs='EPSG:4326')

# 2. 버스 정류장 GeoDataFrame 불러오기 (이미 geometry 포함되어 있어야 함)
gdf_bus = gpd.GeoDataFrame(bus, geometry='geometry', crs='EPSG:4326')

# 3. 거리 단위 계산을 위해 좌표계 변환 (EPSG:5181: meter-based)
gdf_merged = gdf_merged.to_crs(epsg=5181)
gdf_bus = gdf_bus.to_crs(epsg=5181)

# 4. 거리 계산 함수 정의
def is_isolated_from_bus(point):
    distances = gdf_bus.geometry.distance(point)
    return not (distances <= 500).any()

# 5. 거리 기준 필터링
gdf_merged['no_bus_500m'] = gdf_merged.geometry.progress_apply(is_isolated_from_bus)
gdf_isolated = gdf_merged[gdf_merged['no_bus_500m'] == True].copy()

# 6. folium 시각화를 위해 좌표계를 위경도(EPSG:4326)로 복원
gdf_isolated = gdf_isolated.to_crs(epsg=4326)

100%|██████████| 24724/24724 [00:07<00:00, 3201.20it/s]


In [44]:
print("500m 이내 버스 없는 주거시설 \n", gdf_isolated[["단지명","도로명"]])

500m 이내 버스 없는 주거시설 
                   단지명      도로명
10277  인천가정2A-1블록행복주택  행복2로 40


**기존 주거 시설에는 모두 버스정류장이 잘 되어있음** (한 곳 제외)